Majority vote from team's CV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob

from tqdm import tqdm
import seaborn as sns
from collections import Counter

import cv2

In [2]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors

In [3]:
from collections import Counter
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize

In [4]:
from collections import defaultdict

In [5]:
import bz2
import pickle
import _pickle as cPickle

def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        cPickle.dump(data, f)
        
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data

In [6]:
#exps = !ls ../input/whale-21-3-2022/ | grep pbz2 | grep fold4
#exps

In [7]:
FOLD=2

In [8]:
exps_all = [
    [
         'exp16v6Augmix_fold0.pbz2',
         'exp16v4sampler_fold0.pbz2',
         'exp16v6AugMixNew4_fold0.pbz2',
         'exp16v6AugMixNew4_fold0_tta.pbz2',
         'exp16v6AugmixB3_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0.pbz2',
         'exp16v6AugmixB4Step2_fold0_remove_background.pbz2',
         'exp16v6AugmixB4_fold0.pbz2',
         'exp16v6AugmixB4_fold0_remove_background.pbz2',
         'exp16v6AugmixFullB4_fold0.pbz2',
         'exp16v6sampler_fold0.pbz2',
         'exp21b6_fold0.pbz2',
         'exp21b6_fold0_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold1.pbz2',
         'exp16v6AugMixNew4_fold1.pbz2',
         'exp16v6AugMixNew4_fold1_tta.pbz2',
         'exp16v6AugmixB3_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1.pbz2',
         'exp16v6AugmixB4Step2_fold1_remove_background.pbz2',
         'exp16v6AugmixB4_fold1.pbz2',
         'exp21b6_fold1_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold2.pbz2',
         'exp16v6AugMixNew4_fold2.pbz2',
         'exp16v6AugMixNew4_fold2_tta.pbz2',
         'exp16v6AugmixB3_fold2.pbz2',
         'exp16v6AugmixB4Full_fold2.pbz2',
         'exp16v6AugmixB4Step2_fold2.pbz2',
         'exp16v6AugmixB4_fold2.pbz2',
         'exp16v6AugmixB4_fold2_remove_background.pbz2',
         'exp21b6_fold2_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold3.pbz2',
         'exp16v6AugMix1536_fold3.pbz2',
         'exp16v6AugmixB3_fold3.pbz2',
         'exp16v6AugmixB4Full_fold3.pbz2',
         'exp16v6AugmixB4Step2_fold3.pbz2',
         'exp16v6AugmixB4_fold3_remove_background.pbz2',
         'exp21b6_fold3_remove_background.pbz2'
    ],
    [
         'exp16v6Augmix_fold4.pbz2',
         'exp16v6AugMix1536_fold4.pbz2',
         'exp16v6AugmixB3_fold4.pbz2',
         'exp16v6AugmixB4Step2_fold4.pbz2',
         'exp16v6AugmixB4_fold4_remove_background.pbz2',
    ]
]

In [9]:
exps = exps_all[FOLD]

In [10]:
fold2 = pd.read_csv("/kaggle/happywhale/input/whale-21-3-2022/fold2.csv")
fold2.shape

(51033, 8)

In [11]:
valid = fold2.query("fold == @FOLD").reset_index()
train = fold2.query("fold != @FOLD").reset_index()

In [12]:
valid_ind = (fold2.fold == FOLD).values
train_ind = (fold2.fold != FOLD).values

In [13]:
train_inddividual_id_lis = train["individual_id"].values
valid_inddividual_id_lis = valid["individual_id"].values

def get_result_each(files):
    DIRNAME ="../input/whale-21-3-2022/"
      
    ret = []
        
    for fname in files:
        print(fname)
        data = decompress_pickle(DIRNAME + fname  )
        
        train_emb = data["emb_val"]#[new_index,:]
        
        train_emb=normalize(train_emb)
        
        valid_preds = train_emb[valid_ind]
        train_preds = train_emb[train_ind]
        
        
        distances = pairwise_distances(valid_preds,train_preds)
                
        K=1000
        predicted_positions = np.argpartition(distances, K, axis=1)[:, :K]

        all_pred_lis = []
        for i in tqdm(range(predicted_positions.shape[0])):
            nearest = [(train_inddividual_id_lis[j], distances[i, j]) for j in predicted_positions[i]]
            nearest.sort(key=lambda x: x[1])

            prediction = [index_id for index_id, d in nearest]

            done = set()
            pred_list = []
            for pred_id in prediction:
                if pred_id in done:
                    continue
                done.add(pred_id)
                pred_list.append(pred_id)
                if len(pred_list)==20:
                    break

            all_pred_lis.append(pred_list)
        
        ret.append(all_pred_lis)
        
    return ret
    

### get result for each model

In [14]:
ret = get_result_each(exps)

exp16v6Augmix_fold2.pbz2


100%|██████████| 8289/8289 [00:07<00:00, 1123.48it/s]


exp16v6AugMixNew4_fold2.pbz2


100%|██████████| 8289/8289 [00:07<00:00, 1121.48it/s]


exp16v6AugMixNew4_fold2_tta.pbz2


...

100%|██████████| 8289/8289 [00:07<00:00, 1100.41it/s]


exp16v6AugmixB4_fold2_remove_background.pbz2


100%|██████████| 8289/8289 [00:07<00:00, 1102.85it/s]


exp21b6_fold2_remove_background.pbz2


100%|██████████| 8289/8289 [00:07<00:00, 1110.83it/s]


In [15]:
len(ret),len(ret[0]),len(ret[0][0])

(9, 8289, 20)

In [16]:
def get_score(weights):
    
    #train_emb
    vote_result = defaultdict(lambda:defaultdict(int))
    
    for preds, w in zip(ret, weights):
        for i in range(len(preds)):
            
            for j,p in enumerate(preds[i]):
                vote_result[i][p] += (w / (j+1))
        
        
    score_lis = []
    pred_ids = []

    for i in range(len(ret[0])):
        pred_list = np.array(sorted(vote_result[i].items(), key=lambda item: item[1])[::-1])[:5,0].tolist()

        if valid_inddividual_id_lis[i] in pred_list:
            s = 1/(pred_list.index(valid_inddividual_id_lis[i]) + 1)
            score_lis.append(s)
        else:
            score_lis.append(0)
    
    return np.mean(score_lis)

In [17]:
# un-weighted result
get_score([1] * len(exps))

0.8584469377086097

optimize by optuna

In [18]:
import optuna

In [19]:
def objective_wrapper(exps):
    def objective(trial):

        weights = [1]
        for i in range(len(exps)-1):
            w=trial.suggest_uniform(exps[i+1], 0, 4)
            weights.append(w)
        
        
        score_lis = get_score(weights)
        score = np.mean(score_lis)
        return score
    
    return objective


In [20]:
%%time

study = optuna.create_study(direction='maximize')
study.optimize(objective_wrapper(exps), timeout=300)

[I 2022-04-17 07:26:22,117] A new study created in memory with name: no-name-af852a21-0949-45f4-8cb5-57de2c3704a3
[I 2022-04-17 07:26:23,438] Trial 0 finished with value: 0.8434974062009893 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 0.9708093049270907, 'exp16v6AugMixNew4_fold2_tta.pbz2': 1.0720344101103727, 'exp16v6AugmixB3_fold2.pbz2': 2.8141159313934283, 'exp16v6AugmixB4Full_fold2.pbz2': 0.734587314371423, 'exp16v6AugmixB4Step2_fold2.pbz2': 0.17848969621414668, 'exp16v6AugmixB4_fold2.pbz2': 2.283816140975054, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.32316176190871637, 'exp21b6_fold2_remove_background.pbz2': 3.6137071281879423}. Best is trial 0 with value: 0.8434974062009893.
[I 2022-04-17 07:26:24,744] Trial 1 finished with value: 0.8612438171070094 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 3.735075352084775, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.7550521498154787, 'exp16v6AugmixB3_fold2.pbz2': 3.955541409961016, 'exp16v6AugmixB4Full_fold2.pbz2': 3.62205659709672

[I 2022-04-17 07:26:42,031] Trial 14 finished with value: 0.8625487594000081 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 3.1476124182065934, 'exp16v6AugMixNew4_fold2_tta.pbz2': 3.9397647487473106, 'exp16v6AugmixB3_fold2.pbz2': 3.4215466664186582, 'exp16v6AugmixB4Full_fold2.pbz2': 2.9668927244745853, 'exp16v6AugmixB4Step2_fold2.pbz2': 2.7792413314979125, 'exp16v6AugmixB4_fold2.pbz2': 2.8610138523615722, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 2.7791674634550008, 'exp21b6_fold2_remove_background.pbz2': 0.7498263423614198}. Best is trial 12 with value: 0.8635842683073954.
[I 2022-04-17 07:26:43,361] Trial 15 finished with value: 0.8608778702698355 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 3.156432990979086, 'exp16v6AugMixNew4_fold2_tta.pbz2': 3.3895072250911698, 'exp16v6AugmixB3_fold2.pbz2': 3.973419297271662, 'exp16v6AugmixB4Full_fold2.pbz2': 3.4659720385779593, 'exp16v6AugmixB4Step2_fold2.pbz2': 3.714388839136966, 'exp16v6AugmixB4_fold2.pbz2': 3.3842215127588258, 'ex

[I 2022-04-17 07:27:00,648] Trial 28 finished with value: 0.8662303454377287 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.9455647739743953, 'exp16v6AugMixNew4_fold2_tta.pbz2': 0.6129349809120046, 'exp16v6AugmixB3_fold2.pbz2': 0.5924413736099018, 'exp16v6AugmixB4Full_fold2.pbz2': 3.159822766909959, 'exp16v6AugmixB4Step2_fold2.pbz2': 0.03271612594392981, 'exp16v6AugmixB4_fold2.pbz2': 2.0434787987208316, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.49953717493743677, 'exp21b6_fold2_remove_background.pbz2': 2.2814838583665717}. Best is trial 18 with value: 0.8679193308400692.
[I 2022-04-17 07:27:01,972] Trial 29 finished with value: 0.8669361000522782 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.378577529020476, 'exp16v6AugMixNew4_fold2_tta.pbz2': 1.2956652198413707, 'exp16v6AugmixB3_fold2.pbz2': 1.5194980358246413, 'exp16v6AugmixB4Full_fold2.pbz2': 3.6697012998987057, 'exp16v6AugmixB4Step2_fold2.pbz2': 0.2876663100674042, 'exp16v6AugmixB4_fold2.pbz2': 2.089252506679771, '

[I 2022-04-17 07:27:19,410] Trial 42 finished with value: 0.8656452326376322 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.262422715766788, 'exp16v6AugMixNew4_fold2_tta.pbz2': 1.4221559215086563, 'exp16v6AugmixB3_fold2.pbz2': 1.1518169019359825, 'exp16v6AugmixB4Full_fold2.pbz2': 3.7990847580046876, 'exp16v6AugmixB4Step2_fold2.pbz2': 0.1832375435722079, 'exp16v6AugmixB4_fold2.pbz2': 2.215544837174834, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 1.0549005635474782, 'exp21b6_fold2_remove_background.pbz2': 3.05303558448017}. Best is trial 18 with value: 0.8679193308400692.
[I 2022-04-17 07:27:20,766] Trial 43 finished with value: 0.8668456186914385 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.6041219258363304, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.903978197952867, 'exp16v6AugmixB3_fold2.pbz2': 0.8548225607477443, 'exp16v6AugmixB4Full_fold2.pbz2': 3.470812582581323, 'exp16v6AugmixB4Step2_fold2.pbz2': 0.8993468074731816, 'exp16v6AugmixB4_fold2.pbz2': 2.688837882698156, 'exp16v

[I 2022-04-17 07:27:38,217] Trial 56 finished with value: 0.8591567137169742 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 3.708564310929766, 'exp16v6AugMixNew4_fold2_tta.pbz2': 1.8687527424328672, 'exp16v6AugmixB3_fold2.pbz2': 1.0614261226272785, 'exp16v6AugmixB4Full_fold2.pbz2': 1.7462025959419532, 'exp16v6AugmixB4Step2_fold2.pbz2': 2.080163650017245, 'exp16v6AugmixB4_fold2.pbz2': 3.703210860645207, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.6524602607919043, 'exp21b6_fold2_remove_background.pbz2': 3.7716614985875436}. Best is trial 18 with value: 0.8679193308400692.
[I 2022-04-17 07:27:39,542] Trial 57 finished with value: 0.8638516909960993 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.592266788376955, 'exp16v6AugMixNew4_fold2_tta.pbz2': 1.686172024350981, 'exp16v6AugmixB3_fold2.pbz2': 0.47115928790676165, 'exp16v6AugmixB4Full_fold2.pbz2': 3.5363081216716408, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.5452707838815078, 'exp16v6AugmixB4_fold2.pbz2': 2.576662705736832, 'exp1

...

[I 2022-04-17 07:30:08,419] Trial 168 finished with value: 0.8682712027988901 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.2268930630617407, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.1573109797843526, 'exp16v6AugmixB3_fold2.pbz2': 0.8263480992583504, 'exp16v6AugmixB4Full_fold2.pbz2': 3.838960010555714, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.4958414714108688, 'exp16v6AugmixB4_fold2.pbz2': 2.9711793004134552, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.2558727152580005, 'exp21b6_fold2_remove_background.pbz2': 1.8566144590337343}. Best is trial 149 with value: 0.8684059194916959.
[I 2022-04-17 07:30:09,756] Trial 169 finished with value: 0.8681043149555636 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.383518899134864, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.164044696400975, 'exp16v6AugmixB3_fold2.pbz2': 0.8930642398769638, 'exp16v6AugmixB4Full_fold2.pbz2': 3.832806281668414, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.5165232211083288, 'exp16v6AugmixB4_fold2.pbz2': 2.9550253905024757, '

[I 2022-04-17 07:30:27,114] Trial 182 finished with value: 0.8685848715164676 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.030986323090327, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.298996532296508, 'exp16v6AugmixB3_fold2.pbz2': 0.7389981063822092, 'exp16v6AugmixB4Full_fold2.pbz2': 3.914749311719629, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.77664185373741, 'exp16v6AugmixB4_fold2.pbz2': 3.10330129595609, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.13708971421017155, 'exp21b6_fold2_remove_background.pbz2': 1.6017276546626564}. Best is trial 175 with value: 0.8686532352113243.
[I 2022-04-17 07:30:28,469] Trial 183 finished with value: 0.8684622190051073 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 2.035679025989087, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.2935358808147535, 'exp16v6AugmixB3_fold2.pbz2': 0.7299829853130679, 'exp16v6AugmixB4Full_fold2.pbz2': 3.9099062357173024, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.624334289739544, 'exp16v6AugmixB4_fold2.pbz2': 3.0999955074969563, 'exp1

[I 2022-04-17 07:30:46,106] Trial 196 finished with value: 0.8683355450999316 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 1.6615260528870528, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.275725559266025, 'exp16v6AugmixB3_fold2.pbz2': 0.6585161171503586, 'exp16v6AugmixB4Full_fold2.pbz2': 3.8255060139321944, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.9236640662092184, 'exp16v6AugmixB4_fold2.pbz2': 3.054953222148149, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.08805497884647923, 'exp21b6_fold2_remove_background.pbz2': 1.5202324857733507}. Best is trial 175 with value: 0.8686532352113243.
[I 2022-04-17 07:30:47,461] Trial 197 finished with value: 0.8684742831865525 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 1.7308488666012525, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.2486154132845297, 'exp16v6AugmixB3_fold2.pbz2': 0.625565068736295, 'exp16v6AugmixB4Full_fold2.pbz2': 3.819217342934073, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.8859280235974385, 'exp16v6AugmixB4_fold2.pbz2': 3.0336628383662134, 

[I 2022-04-17 07:31:04,887] Trial 210 finished with value: 0.8686391603329714 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 1.9659676731832865, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.128463749109179, 'exp16v6AugmixB3_fold2.pbz2': 0.7109203345511653, 'exp16v6AugmixB4Full_fold2.pbz2': 3.8593086353313177, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.8874664525875076, 'exp16v6AugmixB4_fold2.pbz2': 2.9438505183283827, 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.1610251468174565, 'exp21b6_fold2_remove_background.pbz2': 1.5852622101692213}. Best is trial 175 with value: 0.8686532352113243.
[I 2022-04-17 07:31:06,228] Trial 211 finished with value: 0.8687718663288696 and parameters: {'exp16v6AugMixNew4_fold2.pbz2': 1.9741011458608575, 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.142945591433017, 'exp16v6AugmixB3_fold2.pbz2': 0.6797818704300601, 'exp16v6AugmixB4Full_fold2.pbz2': 3.9964604130617207, 'exp16v6AugmixB4Step2_fold2.pbz2': 1.8954797907800192, 'exp16v6AugmixB4_fold2.pbz2': 2.9474486633265937,

CPU times: user 5min, sys: 1.09 s, total: 5min 1s
Wall time: 5min


In [21]:
study.trials_dataframe().sort_values("value")

,number,value,datetime_start,datetime_complete,duration,params_exp16v6AugMixNew4_fold2.pbz2,params_exp16v6AugMixNew4_fold2_tta.pbz2,params_exp16v6AugmixB3_fold2.pbz2,params_exp16v6AugmixB4Full_fold2.pbz2,params_exp16v6AugmixB4Step2_fold2.pbz2,params_exp16v6AugmixB4_fold2.pbz2,params_exp16v6AugmixB4_fold2_remove_background.pbz2,params_exp21b6_fold2_remove_background.pbz2,state
0,0,0.843497,2022-04-17 07:26:22.120764,2022-04-17 07:26:23.437901,0 days 00:00:01.317137,0.970809,1.072034,2.814116,0.734587,0.178490,2.283816,0.323162,3.613707,COMPLETE
7,7,0.844812,2022-04-17 07:26:31.453249,2022-04-17 07:26:32.764296,0 days 00:00:01.311047,2.111370,0.846310,2.277342,0.234850,3.173974,3.953537,1.753518,1.441102,COMPLETE
5,5,0.846608,2022-04-17 07:26:28.700016,2022-04-17 07:26:29.998710,0 days 00:00:01.298694,0.439622,1.149550,2.670389,2.868654,3.516953,3.747947,3.094981,2.387526,COMPLETE
133,133,0.851027,2022-04-17 07:29:20.048538,2022-04-17 07:29:21.383782,0 days 00:00:01.335244,2.606597,1.314766,1.463435,0.152541,1.201996,2.025432,2.356981,1.509676,COMPLETE
4,4,0.852582,2022-04-17 07:26:27.388785,2022-04-17 07:26:28.698526,0 days 00:00:01.309741,0.993337,1.097395,2.994419,1.851528,1.220914,0.001748,3.277270,2.000092,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,182,0.868585,2022-04-17 07:30:25.781326,2022-04-17 07:30:27.114137,0 days 00:00:01.332811,2.030986,2.298997,0.738998,3.914749,1.776642,3.103301,0.137090,1.601728,COMPLETE
187,187,0.868589,2022-04-17 07:30:32.522178,2022-04-17 07:30:33.984710,0 days 00:00:01.462532,1.964391,2.378099,0.723068,3.939084,1.794860,3.093336,0.100736,1.571370,COMPLETE
210,210,0.868639,2022-04-17 07:31:03.552579,2022-04-17 07:31:04.886826,0 days 00:00:01.334247,1.965968,2.128464,0.710920,3.859309,1.887466,2.943851,0.161025,1.585262,COMPLETE
175,175,0.868653,2022-04-17 07:30:16.446762,2022-04-17 07:30:17.773384,0 days 00:00:01.326622,2.103332,2.284164,0.752474,3.876273,1.393683,2.666470,0.010052,1.620845,COMPLETE


In [22]:
study.best_params

{'exp16v6AugMixNew4_fold2.pbz2': 1.9741011458608575,
 'exp16v6AugMixNew4_fold2_tta.pbz2': 2.142945591433017,
 'exp16v6AugmixB3_fold2.pbz2': 0.6797818704300601,
 'exp16v6AugmixB4Full_fold2.pbz2': 3.9964604130617207,
 'exp16v6AugmixB4Step2_fold2.pbz2': 1.8954797907800192,
 'exp16v6AugmixB4_fold2.pbz2': 2.9474486633265937,
 'exp16v6AugmixB4_fold2_remove_background.pbz2': 0.1576716329922889,
 'exp21b6_fold2_remove_background.pbz2': 1.585196642019381}

In [23]:
best_weights = list(study.best_params.values())

In [24]:
# weighted result
get_score([1] + best_weights)

0.8687718663288696

In [25]:
# un-weighted result
get_score([1] * len(exps))

0.8584469377086097